# PS6: Let's Build a Multiple Asset Synthetic Data Generation System 
In this problem set, you will build a synthetic data generation system for multiple assets. The goal is to create a framework that allows you to simulate the price movements of various financial assets over time, taking into account correlations between them. 

> __Learning Objectives:__
> 
> By the end of this problem set, you should be able to:
> Three leaning objectives here

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

### Data
We use daily open-high-low-close (OHLC) data for firms in the [S&P 500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2025` until `11-18-2025`, along with data for exchange-traded funds and volatility products.

Load the `original_dataset::DataFrame` by calling [the `MyTestingMarketDataSet()` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/data/#VLQuantitativeFinancePackage.MyTestingMarketDataSet).

In [2]:
original_dataset = MyTestingMarketDataSet() |> x-> x["dataset"] # load the original dataset (testing)

Dict{String, DataFrame} with 483 entries:
  "NI"   => 220×8 DataFrame…
  "EMR"  => 220×8 DataFrame…
  "CTAS" => 220×8 DataFrame…
  "HSIC" => 220×8 DataFrame…
  "KIM"  => 220×8 DataFrame…
  "PLD"  => 220×8 DataFrame…
  "IEX"  => 220×8 DataFrame…
  "BAC"  => 220×8 DataFrame…
  "CBOE" => 220×8 DataFrame…
  "EXR"  => 220×8 DataFrame…
  "NCLH" => 220×8 DataFrame…
  "CVS"  => 220×8 DataFrame…
  "DRI"  => 220×8 DataFrame…
  "DTE"  => 220×8 DataFrame…
  "ZION" => 220×8 DataFrame…
  "AVY"  => 220×8 DataFrame…
  "EW"   => 220×8 DataFrame…
  "EA"   => 220×8 DataFrame…
  "NWSA" => 220×8 DataFrame…
  ⋮      => ⋮

Not all tickers have the maximum number of trading days due to acquisition or delisting events. Collect only tickers with the maximum number of trading days.

Compute the number of records for `AAPL` and save that value in the `maximum_number_trading_days::Int64` variable:

In [3]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow # maximum number of trading days in our dataset

220

Iterate through the data and collect only tickers with `maximum_number_trading_days` records. Save the data in the `dataset::Dict{String,DataFrame}` variable:

In [4]:
dataset = let

    # initialize -
    dataset = Dict{String, DataFrame}();

    # iterate through the dictionary; we can't guarantee a particular order
    for (ticker, data) ∈ original_dataset  # we get each (K, V) pair!
        if (nrow(data) == maximum_number_trading_days) # check if ticker has maximum trading days
            dataset[ticker] = data;
        end
    end
    dataset; # return
end;

Get a list of firms in the cleaned dataset and sort them alphabetically. Store the sorted ticker symbols in the `list_of_tickers_price_data::Array{String,1}` variable:

In [5]:
list_of_tickers_price_data = keys(dataset) |> collect |> sort; # list of tickers in our dataset

Next, let's load the HMM model that we estimated using historical data (2014 - 2024). First, we specify the `path_to_save_file::String` variable:

In [6]:
path_to_save_file = joinpath(_PATH_TO_DATA,"HMM-WJ-SPY-N-100-daily-aggregate.jld2");

Load the [`HDF5` binary file](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) using [the `load(...)` function from JLD2.jl](https://github.com/JuliaIO/JLD2.jl.git). This loads the saved data into the `saved_state_dict::Dict{String, Any}` variable.

In [7]:
saved_state_dict = load(path_to_save_file)

Dict{String, Any} with 11 entries:
  "risk_free_rate"          => 0.043
  "model"                   => MyHiddenMarkovModel([1, 2, 3, 4, 5, 6, 7, 8, 9, …
  "decode"                  => Dict{Int64, Normal}(5=>Normal{Float64}(μ=-3.3565…
  "encoded_archive_with_ju… => [53 72 … 82 92; 93 82 … 80 5; … ; 14 67 … 87 13;…
  "insampledataset"         => [-0.62754, 0.839626, 0.162992, -0.0111337, 0.257…
  "jump_model"              => MyHiddenMarkovModelWithJumps([1, 2, 3, 4, 5, 6, …
  "encoded_archive"         => [48 29 … 28 33; 42 10 … 10 68; … ; 18 37 … 39 77…
  "number_of_states"        => 100
  "in_sample_decoded_archi… => [0.202562 0.962389 … 1.56531 2.7557; 2.94573 1.6…
  "stationary"              => Categorical{Float64, Vector{Float64}}(…
  "in_sample_decoded_archi… => [0.0753184 -0.675607 … -0.759002 -0.470222; -0.1…

Finally, let's load the single index model parameters that we computed in the previous example. We'll store this data in the `sim_model_parameters::Dict{String,NamedTuple}` variable. In addition, we return a few other useful variables, such as the historical market growth rate, the mean and variance of the market growth, etc.

In [8]:
sim_model_parameters,Gₘ,Ḡₘ,Varₘ = let

    # initialize -
    path_to_sim_model_parameters = joinpath(_PATH_TO_DATA,"SIMs-SPY-SP500-01-03-14-to-12-31-24.jld2");
    sim_model_parameters = JLD2.load(path_to_sim_model_parameters);
    parameters = sim_model_parameters["data"]; # return

    Gₘ = sim_model_parameters["Gₘ"]; # Get the past market growth rate 
    Ḡₘ = sim_model_parameters["Ḡₘ"]; # mean of market growth rates
    Varₘ = sim_model_parameters["Varₘ"]; # variance of market growth

    # return -
    parameters,  Gₘ , Ḡₘ, Varₘ;
end;

Now let's get a list of all tickers for which we have single index model parameters:

In [9]:
tickers_that_we_sim_sim_data_for = keys(sim_model_parameters) |> collect |> sort;

We need to use only the tickers for which we have both price data and SIM parameters. We'll compute [the intersection of the two lists](https://docs.julialang.org/en/v1/base/collections/#Base.intersect) and store the result in the `list_of_tickers::Array{String,1}` variable:

In [10]:
list_of_tickers = intersect(tickers_that_we_sim_sim_data_for, list_of_tickers_price_data);

### Constants and Parameters
Finally, we set constants and parameters needed for the problem set. See the comments in the code for additional information on each parameter, units, permissible values, etc.

In [11]:
Δt = (1/252); # time step 1 x trading in units of years
number_of_paths = 10000; # number of potential futures should we look at
blue_color = colorant"rgb(68,152,242)";
myticker = "SPY"; # ticker symbol for the SPY ETF
number_of_states = saved_state_dict["number_of_states"]; # number of hidden states in the HMM
risk_free_rate = saved_state_dict["risk_free_rate"]; # risk-free interest rate (annualized)
π̄ = saved_state_dict["stationary"]; # stationary distribution of the Markov chain

___

## Task 1: Generate Synthetic Growth Rate Trajectories for SPY
In this task, we will generate synthetic growth rate trajectories for the SPY ETF using the HMM parameters we estimated earlier. First, we'll generated encoded hidden state sequences for each synthetic path. Then, we'll decode these hidden state sequences to generate observable growth rates for each path.

Let's start by generating the encoded hidden state sequences for each synthetic path. We'll store the results in the `encoded_hidden_state_matrix::Array{Int64,2}` variable, where each row corresponds to a synthetic path and each column corresponds to a time step.

In [15]:
market_factor_encoded_archive, market_jump_indicator_archive = let
    
    model = saved_state_dict["jump_model"]; # grab whichever version of the model you want
    number_of_steps = 2*(maximum_number_trading_days - 1); # we'll generate paths twice as long as the original data
    encoded_archive = Array{Int64,2}(undef, number_of_steps, number_of_paths);
    jump_indicator_archive = Array{Int64,2}(undef, number_of_steps, number_of_paths);
    
    for i ∈ 1:number_of_paths
        start_state = rand(π̄);
        tmp = model(start_state, number_of_steps) # generates state sequence of length number_of_steps
        for j ∈ 1:number_of_steps
            encoded_archive[j,i] = tmp[j,1]
            jump_indicator_archive[j,i] = tmp[j,2]
        end
    end
    encoded_archive, jump_indicator_archive
end

([43 19 … 24 98; 28 42 … 24 27; … ; 77 31 … 88 21; 52 89 … 49 66], [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])

__How many jumps?__ The `jump_indicator_archive::Array{Int64,2}` array holds jump indicators for each time step and sample path. A value of `1` indicates a jump, while `0` indicates no jump.

In [ ]:
jump_trial_index_set = let
    
    (number_of_steps, number_of_paths) = size(market_factor_encoded_archive);
    number_of_steps = 2*length(Rᵢ);
    has_jumps_flag = any(x->x==1, jump_indicator_archive);
    jump_trial_index_set = Set{Int64}();
    if (has_jumps_flag == false)
        println("No jumps were detected in the simulated paths.")
    end;

    for i ∈ 1:number_of_paths
        for j ∈ 1:number_of_steps
            if (jump_indicator_archive[j,i] == 1)
                push!(jump_trial_index_set, i);
            end
        end
    end
    println("Number of paths with at least one jump event: $(length(jump_trial_index_set)) out of $number_of_paths total paths.")

    jump_trial_index_set; # return the trial index set
end;

## Summary
One direct summary sentence goes here.

> __Key Takeaways:__
> 
> Three key takeaways go here.

One direct concluding sentence goes here.
___

## Tests
The code block below shows how we implemented the tests and what we are testing. In these tests, we check values in your notebook and give feedback on which items are correct, missing, etc.

In [12]:
@testset verbose = true "CHEME 5800 PS6 Test Suite" begin
end

Test Summary:             | Total  Time
CHEME 5800 PS6 Test Suite |     0  0.1s


Test.DefaultTestSet("CHEME 5800 PS6 Test Suite", Any[], 0, false, true, true, 1.763742609642404e9, 1.763742609784168e9, false, "/Users/jeffreyvarner/Desktop/julia_work/CHEME-5660-Fall-2025/PS6-CHEME-5660-TEMPLATE-Fall-2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl", Random.Xoshiro(0x934e0a032d4cf8d3, 0x865dc7b2e5bd0de4, 0x6e89da2f45cbc12d, 0xbda7397a289eac74, 0x7eab365f57d59f8c))

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___